In [25]:
1+1

2

In [44]:
proto_string = """
syntax = "proto3";
package helloworld;

service Greeter {
    rpc SayHello (HelloRequest) returns (HelloReply);
    rpc SendVoice(stream VoiceRequest) returns (Empty);
    rpc GetVoice(Empty) returns (stream VoiceReply);
    rpc GetCurrentUsersUUID(Empty) returns (CurrentUsersUUIDReply);
    rpc StartSpeaking(StartSpeakingRequest) returns (Empty);
    rpc StopSpeaking(StopSpeakingRequest) returns (Empty);
}

message Empty {
}

message VoiceRequest {
    string uuid = 1;
    int64 timestamp = 2;
    bytes voice = 3;
}

message VoiceReply {
    string uuid = 1;
    int64 timestamp = 2;
    bytes voice = 3;
}

message CurrentUsersUUIDReply {
    repeated string uuid = 1;
}

message StartSpeakingRequest {
    string uuid = 1;
}

message StopSpeakingRequest {
    string uuid = 1;
}

message HelloRequest {
   string name = 1;
}

message HelloReply {
    string message = 1;
}
"""

In [45]:
import re

In [46]:
found = re.findall(r"message\s+(?P<object_name>\w+)\s+\{(?P<properties>(\s*.*?\s*)+)\}", proto_string, re.DOTALL)
found = [
            [" " + string.strip() for string in one][:2]
            for one in found
        ]
found

[[' Empty', ' '],
 [' VoiceRequest',
  ' string uuid = 1;\n    int64 timestamp = 2;\n    bytes voice = 3;'],
 [' VoiceReply',
  ' string uuid = 1;\n    int64 timestamp = 2;\n    bytes voice = 3;'],
 [' CurrentUsersUUIDReply', ' repeated string uuid = 1;'],
 [' StartSpeakingRequest', ' string uuid = 1;'],
 [' StopSpeakingRequest', ' string uuid = 1;'],
 [' HelloRequest', ' string name = 1;'],
 [' HelloReply', ' string message = 1;']]

In [47]:
found_2 = []
for one in found.copy():
    if len(one) == 2:
        class_name = one[0]
        property_text = one[1]
        if len(property_text) == 0:
            continue
        #property_list = re.findall(r"\w+\s+(?P<property>\w+)\s+=\s+\d+;", property_text)
        # print(class_name, property_list)
        property_list = re.findall(r"(?P<feature>\w*)\s+(?P<type>\w+)\s+(?P<property>\w+)\s+=\s+\d+;", property_text)
        found_2.append([class_name.strip(), property_list])
        print(found_2[-1])

['Empty', []]
['VoiceRequest', [('', 'string', 'uuid'), ('', 'int64', 'timestamp'), ('', 'bytes', 'voice')]]
['VoiceReply', [('', 'string', 'uuid'), ('', 'int64', 'timestamp'), ('', 'bytes', 'voice')]]
['CurrentUsersUUIDReply', [('repeated', 'string', 'uuid')]]
['StartSpeakingRequest', [('', 'string', 'uuid')]]
['StopSpeakingRequest', [('', 'string', 'uuid')]]
['HelloRequest', [('', 'string', 'name')]]
['HelloReply', [('', 'string', 'message')]]


In [48]:
import json

defination_tree = {}
for one in found_2.copy():
    class_name = one[0]
    variable_list = one[1]

    defination_tree[class_name] = {}
    for one_variable in variable_list:
        feature = one_variable[0]
        type = one_variable[1]
        name = one_variable[2]
        defination_tree[class_name][name] = {
            "type": type, 
            "nullable": True if feature == "optional" else False,
            "is_list": True if feature == "repeated" else False
        }

print(json.dumps(defination_tree, indent=4, sort_keys=False))

{
    "Empty": {},
    "VoiceRequest": {
        "uuid": {
            "type": "string",
            "nullable": false,
            "is_list": false
        },
        "timestamp": {
            "type": "int64",
            "nullable": false,
            "is_list": false
        },
        "voice": {
            "type": "bytes",
            "nullable": false,
            "is_list": false
        }
    },
    "VoiceReply": {
        "uuid": {
            "type": "string",
            "nullable": false,
            "is_list": false
        },
        "timestamp": {
            "type": "int64",
            "nullable": false,
            "is_list": false
        },
        "voice": {
            "type": "bytes",
            "nullable": false,
            "is_list": false
        }
    },
    "CurrentUsersUUIDReply": {
        "uuid": {
            "type": "string",
            "nullable": false,
            "is_list": true
        }
    },
    "StartSpeakingRequest": {
        "uuid": {
  

In [2]:
import re
from typing import Any

Null_value_identify_symbol = "xx"

def replace_null_to_null_identify_symbol(json_string: str) -> str:
    def try_to_do_a_replace(match_obj: Any):
        if match_obj.group() is not None:
            before = match_obj.group('before')
            null = match_obj.group('null')
            after = match_obj.group('after')
            return f'{before}"{Null_value_identify_symbol}"{after}'
    result = re.sub(r"(?P<before>\"\w+?\"\s*:\s*)(?P<null>null)(?P<after>,?)", try_to_do_a_replace, json_string, flags=re.MULTILINE) #type: ignore
    return result #type: ignore

result = replace_null_to_null_identify_symbol("""
{
    "Okok": {
        "Option": [
            null, 
            null
        ]
    }, 
    "Page_size": "2", 
    "Page_number": {
        "ok": null,
        "no": null
    }
}
""")
print(result)


{
    "Okok": {
        "Option": [
            null, 
            null
        ]
    }, 
    "Page_size": "2", 
    "Page_number": {
        "ok": "xx",
        "no": "xx"
    }
}

